In [1]:
#Importing required libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import sklearn

In [2]:
credit_card_data = pd.read_csv('creditcard.csv')
credit_card_data.head(5)
credit_card_data.tail(5)

# time = time elapsed from first transaction
# V1 - V28 = feature of each transaction (hideen through PCA)
# Amount (USD) - value in USD of a transaction
# Class = Fraud/legit - 0 = legit, 1 = fraud
# 284806 data points

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
284802,172786.0,-11.881118,10.071785,-9.834783,-2.066656,-5.364473,-2.606837,-4.918215,7.305334,1.914428,...,0.213454,0.111864,1.014480,-0.509348,1.436807,0.250034,0.943651,0.823731,0.77,0
284803,172787.0,-0.732789,-0.055080,2.035030,-0.738589,0.868229,1.058415,0.024330,0.294869,0.584800,...,0.214205,0.924384,0.012463,-1.016226,-0.606624,-0.395255,0.068472,-0.053527,24.79,0
284804,172788.0,1.919565,-0.301254,-3.249640,-0.557828,2.630515,3.031260,-0.296827,0.708417,0.432454,...,0.232045,0.578229,-0.037501,0.640134,0.265745,-0.087371,0.004455,-0.026561,67.88,0
284805,172788.0,-0.240440,0.530483,0.702510,0.689799,-0.377961,0.623708,-0.686180,0.679145,0.392087,...,0.265245,0.800049,-0.163298,0.123205,-0.569159,0.546668,0.108821,0.104533,10.00,0
284806,172792.0,-0.533413,-0.189733,0.703337,-0.506271,-0.012546,-0.649617,1.577006,-0.414650,0.486180,...,0.261057,0.643078,0.376777,0.008797,-0.473649,-0.818267,-0.002415,0.013649,217.00,0


In [3]:
# Obtaining Information
credit_card_data.info()
# All of Float data types (except for Class - Int)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 284807 entries, 0 to 284806
Data columns (total 31 columns):
 #   Column  Non-Null Count   Dtype  
---  ------  --------------   -----  
 0   Time    284807 non-null  float64
 1   V1      284807 non-null  float64
 2   V2      284807 non-null  float64
 3   V3      284807 non-null  float64
 4   V4      284807 non-null  float64
 5   V5      284807 non-null  float64
 6   V6      284807 non-null  float64
 7   V7      284807 non-null  float64
 8   V8      284807 non-null  float64
 9   V9      284807 non-null  float64
 10  V10     284807 non-null  float64
 11  V11     284807 non-null  float64
 12  V12     284807 non-null  float64
 13  V13     284807 non-null  float64
 14  V14     284807 non-null  float64
 15  V15     284807 non-null  float64
 16  V16     284807 non-null  float64
 17  V17     284807 non-null  float64
 18  V18     284807 non-null  float64
 19  V19     284807 non-null  float64
 20  V20     284807 non-null  float64
 21  V21     28

In [4]:
# fraud vs legit
credit_card_data['Class'].value_counts()
# 492 Fraudulent, remaining are legit
# too little fraudulent - imbalanced dataset, will likely result in many false legit

0    284315
1       492
Name: Class, dtype: int64

In [5]:
# Separation of class
legit = credit_card_data[credit_card_data.Class == 0]
fraud = credit_card_data[credit_card_data.Class == 1]

In [6]:
# Basic statistitcal Information
legit.Amount.describe()
# mean: 88.29
# std: 250.11
# median: 22
# max: 25691.16

count    284315.000000
mean         88.291022
std         250.105092
min           0.000000
25%           5.650000
50%          22.000000
75%          77.050000
max       25691.160000
Name: Amount, dtype: float64

In [7]:
fraud.Amount.describe()
# mean: 122.21
# std: 256,68
# Max: 2125.87

count     492.000000
mean      122.211321
std       256.683288
min         0.000000
25%         1.000000
50%         9.250000
75%       105.890000
max      2125.870000
Name: Amount, dtype: float64

In [8]:
# Comparing the values for fraud vs legit
credit_card_data.groupby('Class').mean()
# Many of the DV have huge differences


,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
Class,,,,,,,,,,,,,,,,,,,,,
0,94838.202258,0.008258,-0.006271,0.012171,-0.007860,0.005453,0.002419,0.009637,-0.000987,0.004467,...,-0.000644,-0.001235,-0.000024,0.000070,0.000182,-0.000072,-0.000089,-0.000295,-0.000131,88.291022
1,80746.806911,-4.771948,3.623778,-7.033281,4.542029,-3.151225,-1.397737,-5.568731,0.570636,-2.581123,...,0.372319,0.713588,0.014049,-0.040308,-0.105130,0.041449,0.051648,0.170575,0.075667,122.211321


In [9]:
# Dealing with uneven dataset: Under-Sampling
# Building sample data set from both legit & fraud, will ensure more realistic distribution
# prevents bias and reduces risk of falsely legit transactios being predicted
# on average, about 10% of transactions are fraudulent, we will replicate this through random sampling

# Randomly sampling 4428 data points - replicating reality
legit_sample = legit.sample(n=4428)

# Concatanating the samples together
new_dataset = pd.concat([legit_sample, fraud], axis=0)
new_dataset['Class'].value_counts()
# Distribution between both legit & fraud transactions mirrors that of the real world
# 4428 real, 492 fraudulent

0    4428
1     492
Name: Class, dtype: int64

In [10]:
# splitting into predictors & target
# X = predictors, Y = target
X = new_dataset.drop(columns='Class', axis=1)
Y = new_dataset['Class']

# train & test splitting
# 20% will be for testing, 80% for train, stratisfy for same distribution, random_state - for replicability
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.2, stratify=Y, random_state=2)

print(X_train.shape, X_test.shape)

(3936, 30) (984, 30)


In [11]:
# Binary Classification - Logistic Regression

# Initializing Model:
model = LogisticRegression(solver='lbfgs')

# Training Model:
model.fit(X_train, Y_train)

# Accuracy (training data)
X_train_predictions = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_predictions, Y_train)

# Score on training data:
print("Your model has an accuracy of: ", training_data_accuracy)

# Very high accuracy, potential for overfitting?
# Will have to check for potential of false legits/frauds

Your model has an accuracy of:  0.978150406504065


In [12]:
# Accuracy (Testing data)
X_test_predictions = model.predict(X_test)
testing_data_accuracy = accuracy_score(X_test_predictions, Y_test)

# Score on testing data:
print("Your model has an accuracy of: ", testing_data_accuracy)

# Still very highly accurate, potential for overfitting
# Need to account of potential of false legits/frauds from having uneven spread of train vs test
# If very large amoutns of false legits, a potential solution is to re-distribute the data into a uniform distribution
# 50/50 split between fraud/legit -> may have less false legits
# In credit card transactions, false legits are particularly harmful, so re-distributing may be more beneficial

Your model has an accuracy of:  0.9817073170731707


In [13]:
# Viewing Distributions:
# Training confusion matrix
print(sklearn.metrics.confusion_matrix(Y_train, X_train_predictions))
# Testing confusion matrix
print(sklearn.metrics.confusion_matrix(Y_test, X_test_predictions))

#Very good at predicting both frauds and non-frauds, accuracy percentage is not skewed towards either side, no requirement to re-distribute the data

[[3513   29]
 [  57  337]]
[[878   8]
 [ 10  88]]
